# Notebook de la práctica 1 de **FAA**.

**Javier López Cano y Gonzalo Madrigal.** Grupo 1462.

In [1]:
# -*- coding: utf-8 -*-

# coding: utf-8
from Datos import Datos
import numpy as np
from Clasificador import Clasificador, ClasificadorNaiveBayes
from EstrategiaParticionado import ValidacionSimple, ValidacionCruzada
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.preprocessing import OneHotEncoder
from sklearn.naive_bayes import GaussianNB, MultinomialNB
from IPython.display import HTML, display

 En esta práctica hemos trabajado con el algoritmo de clasificación **Naive Bayes**. Hemos experimentado con los **modelos de partición** de los datos para la validación de resultados; hemos aplicado la corrección de Laplace al algoritmo para observar la mejora y finalmente hemos comparado los resultados obtenidos con los resultantes de ejecutar los métodos de la libreria **Scikit-Learn**. Por último, para un análisis más exhaustivo de los errores de clasificación hemos realizado una **matriz de confusión** y **la curva ROC**.

Función que compara una lista predicción y los datos reales correspondientes para calcular el error del **clasificador**:

In [2]:
def error_sk(data, pred):
    tot = 0
    for i in range(0, len(pred)):
        if pred[i] != data[i]:
            tot += 1
    tot /= len(pred)
    #print("error de: " + str(tot))
    return tot


Realizamos la prueba con dos ficheros.
    

En primer lugar realizaremos pruebas con el fihero *Tic Tac Toe*:

In [3]:
    tictac = Datos("tic-tac-toe.data")
    german = Datos("german.data")
    
    print("Tic-tac-toe:\n")
    print("nominalAtributos:")
    print(tictac.nominalAtributos)
    print("\nDiccionario:")
    print(tictac.diccionario)
    print("\nDatos:")
    print(tictac.datos)

Tic-tac-toe:

nominalAtributos:
[True, True, True, True, True, True, True, True, True, True]

Diccionario:
{'TLeftSq': {'b': 0, 'o': 1, 'x': 2}, 'TMidSq': {'b': 0, 'o': 1, 'x': 2}, 'TRightSq': {'b': 0, 'o': 1, 'x': 2}, 'MLeftSq': {'b': 0, 'o': 1, 'x': 2}, 'MMidSq': {'b': 0, 'o': 1, 'x': 2}, 'MRightSq': {'b': 0, 'o': 1, 'x': 2}, 'BLeftSq': {'b': 0, 'o': 1, 'x': 2}, 'BMidSq': {'b': 0, 'o': 1, 'x': 2}, 'BRightSq': {'b': 0, 'o': 1, 'x': 2}, 'Class': {'negative': 0, 'positive': 1}}

Datos:
[[2 2 2 ... 1 1 1]
 [2 2 2 ... 2 1 1]
 [2 2 2 ... 1 2 1]
 ...
 [1 2 1 ... 1 2 0]
 [1 2 1 ... 1 2 0]
 [1 1 2 ... 2 2 0]]


Probamos nuestro clasificador **Naive-Bayes** con validación simple. Este tipo de particionado divide la lista en dos partes aleatorias de un tamaño determinado, una para entrenar y otra para probar las predicciones que hemos hecho. realizamos este proceso y las respectivas pruebas 100 veces y nos quedamos con los valores medios de error, ya que de hacerlo una única vez el resultado estaría demasiado sesgado por el resultado de la separación aleatoria. La principal ventaja respecto a la validación cruzada es que de esta forma la división de datos entrenamiento-prueba no depende del número de iteraciones. Por otro lado con este método hay algunas muestras que pueden quedar sin evaluar y otras que pueden ser evaluadas más de una vez.



Obtenemos dos resultados: con y sin **corrección de Laplace**. La corrección de laplace nos evita caer en el error de hacerle "creer" a Naive-Bayes que porque no tengamos ningún ejemplo de pertenencia a una clase dados una serie de valores, no significa que la probabilidad sea automáticamente 0. **Suaviza el efecto de los ceros**.

In [4]:
    print("\n\nTicTacToe:")
    print("\nValidación Simple")

    print("\nValidando 100 veces con clasificador propio:")
    error = 0
    error_lap = 0
    NB_laplace = 0.0
    NB_noLap = 0.0
    for _ in range(0, 100):
        nb = ClasificadorNaiveBayes()
        vs = ValidacionSimple()
        ret = Clasificador.validacion(vs, tictac, nb)
        error += ret[0]
        error_lap += ret[1]
    NB_laplace = error/100
    NB_noLap = error_lap / 100
    print("Error medio sin laplace " + str(NB_laplace))
    print("Error medio con laplace " + str(NB_noLap))




TicTacToe:

Validación Simple

Validando 100 veces con clasificador propio:
Error medio sin laplace 0.3006315789473683
Error medio con laplace 0.2478947368421054


Repetimos la misma prueba con MultinomialNV, método del paquete **scickit-learn** para comparar resultados

In [5]:
    print("\nValidando 100 veces con MultinomialNB:")
    enc = OneHotEncoder(sparse=False)
    tictac.datos = enc.fit_transform(tictac.datos)
    atr = tictac.datos[:, :len(tictac.nominalAtributos) - 1]
    clase = tictac.datos[:, len(tictac.nominalAtributos) - 1]
    error = 0
    Multi = 0.0
    for _ in range(0, 100):
        train_x, test_x, train_y, test_y = train_test_split(atr, clase, test_size=0.1)
        nb = MultinomialNB()
        nb.fit(train_x, train_y)
        res = nb.predict(test_x)
        error += error_sk(test_y, res)
    multi = error/100
    print("Error medio " + str(multi))


Validando 100 veces con MultinomialNB:
Error medio 0.26177083333333334


Podemos observar que el valor medio obtenido es semejante al que nos salió de nuestra implementación con correción de **Laplace**.

Repetimos la misma prueba con GaussianNB, método del paquete **scickit-learn** para comparar resultados

In [6]:

    print("\nValidando 100 veces con GaussianNB:")
    enc.inverse_transform(tictac.datos)
    atr = tictac.datos[:, :len(tictac.nominalAtributos) - 1]
    clase = tictac.datos[:, len(tictac.nominalAtributos) - 1]
    error = 0
    gauss = 0.0
    for _ in range(0, 100):
        train_x, test_x, train_y, test_y = train_test_split(atr, clase, test_size=0.1)
        nb = GaussianNB()
        nb.fit(train_x, train_y)
        res = nb.predict(test_x)
        error += error_sk(test_y, res)
    gauss = error / 100
    print("Error medio " + str(gauss))




Validando 100 veces con GaussianNB:
Error medio 0.2585416666666666


In [7]:
salida = "<table><tr><th>Clasificador</th><th>Nuestro</th><th>MultinomialNB</th><th>Gausssian</th></tr><tr><td>NB sin Laplace</td>"
salida += "<td>%f</td>" % (NB_laplace)
salida += "<td>-</td>" 
salida += "<td>-</td></tr>" 
salida += "<tr><td>NB con Laplace</td>"
salida += "<td>%f</td>" % (NB_noLap)
salida += "<td>%f</td>" % (multi)
salida += "<td>%f</td></tr></table>" % (gauss)

In [8]:
display(HTML(salida))

Clasificador,Nuestro,MultinomialNB,Gausssian
NB sin Laplace,0.300632,-,-
NB con Laplace,0.247895,0.261771,0.258542


Ahora vamos a cambiar de estrategia de particionado: usaremos la validación cruzada. En este caso lo que hacemos es "partir" la lista de valores en k sublistas. Iteramos de tal forma que cada vez la sublista de test es una y las de train serán las demás.  Finalmente se hará la media entre los valores obtenidos en las pruebas para calcular el error promedio. La principal ventaja de este método es que el valor que con una única ejecución obtenemos valores fiables y resistentes a sesgos; además aseguramos que todos los datos habrán sido considerados tanto para entrenar como para validar.

In [9]:
    print("\nValidacion Cruzada")
    print("\nValidando con clasificador propio:")
    tictac = Datos("tic-tac-toe.data")
    nb = ClasificadorNaiveBayes()
    vc = ValidacionCruzada()
    error = Clasificador.validacion(vc, tictac, nb)
    NB_laplace = error[1]
    NB_noLap = error[0]
    print("Error medio sin laplace " + str(NB_noLap))
    print("Error medio con laplace " + str(NB_laplace))


Validacion Cruzada

Validando con clasificador propio:
Error en partición 0 sin Laplace: 0.3368421052631579
Error en partición 0 con Laplace: 0.2631578947368421
Error en partición 1 sin Laplace: 0.3157894736842105
Error en partición 1 con Laplace: 0.3157894736842105
Error en partición 2 sin Laplace: 0.23157894736842105
Error en partición 2 con Laplace: 0.18947368421052632
Error en partición 3 sin Laplace: 0.3157894736842105
Error en partición 3 con Laplace: 0.25263157894736843
Error en partición 4 sin Laplace: 0.3473684210526316
Error en partición 4 con Laplace: 0.25263157894736843
Error en partición 5 sin Laplace: 0.2631578947368421
Error en partición 5 con Laplace: 0.25263157894736843
Error en partición 6 sin Laplace: 0.23157894736842105
Error en partición 6 con Laplace: 0.15789473684210525
Error en partición 7 sin Laplace: 0.37894736842105264
Error en partición 7 con Laplace: 0.3684210526315789
Error en partición 8 sin Laplace: 0.2631578947368421
Error en partición 8 con Laplace: 0

Repetimos la misma prueba con MultinomialNB, método del paquete **scickit-learn** para comparar resultados

In [10]:
    print("\nValidando con MultinomialNB:")
    enc = OneHotEncoder(sparse=False)
    tictac.datos = enc.fit_transform(tictac.datos)
    atr = tictac.datos[:, :len(tictac.nominalAtributos) - 1]
    clase = tictac.datos[:, len(tictac.nominalAtributos) - 1]
    acierto = cross_val_score(MultinomialNB(), atr, clase)
    error = []
    for data in acierto:
        error.append(1 - data)
    print("Error por particiones: ")
    print(error)
    multi = sum(error) / len(error)
    print("Error medio: " + str(multi))



Validando con MultinomialNB:
Error por particiones: 
[0.26041666666666663, 0.26041666666666663, 0.26041666666666663, 0.26178010471204194, 0.26178010471204194]
Error medio: 0.26096204188481675


Repetimos la misma prueba con GaussianNB, método del paquete **scickit-learn** para comparar resultados.


In [11]:
    print("\nValidando con GaussianNB:")
    enc.inverse_transform(tictac.datos)
    atr = tictac.datos[:, :len(tictac.nominalAtributos) - 1]
    clase = tictac.datos[:, len(tictac.nominalAtributos) - 1]
    acierto = cross_val_score(GaussianNB(), atr, clase)
    error = []
    for data in acierto:
        error.append(1 - data)
    print("Error por particiones: ")
    print(error)
    gauss = sum(error) / len(error)
    print("Error medio: " + str(gauss))


Validando con GaussianNB:
Error por particiones: 
[0.26041666666666663, 0.26041666666666663, 0.49479166666666663, 0.40314136125654454, 0.26178010471204194]
Error medio: 0.3361092931937173


In [12]:
salida = "<table><tr><th>Clasificador</th><th>Nuestro</th><th>MultinomialNB</th><th>Gausssian</th></tr><tr><td>NB sin Laplace</td>"
salida += "<td>%f</td>" % (NB_noLap)
salida += "<td>-</td>" 
salida += "<td>-</td></tr>" 
salida += "<tr><td>NB con Laplace</td>"
salida += "<td>%f</td>" % (NB_laplace)
salida += "<td>%f</td>" % (multi)
salida += "<td>%f</td></tr></table>" % (gauss)

In [13]:
display(HTML(salida))

Clasificador,Nuestro,MultinomialNB,Gausssian
NB sin Laplace,0.302105,-,-
NB con Laplace,0.254737,0.260962,0.336109


Ahora vamos a hacer las mismas pruebas con los datos del fichero *german*

In [14]:
    print("\n\nGerman:\n")
    print("nominalAtributos:")
    print(german.nominalAtributos)
    print("\nDiccionario:")
    print(german.diccionario)
    print("\nDatos:")
    print(german.datos)



German:

nominalAtributos:
[True, False, True, True, False, True, True, False, True, True, False, True, False, True, True, False, True, False, True, True, True]

Diccionario:
{'A1': {'A11': 0, 'A12': 1, 'A13': 2, 'A14': 3}, 'A2': {}, 'A3': {'A30': 0, 'A31': 1, 'A32': 2, 'A33': 3, 'A34': 4}, 'A4': {'A40': 0, 'A41': 1, 'A410': 2, 'A42': 3, 'A43': 4, 'A44': 5, 'A45': 6, 'A46': 7, 'A48': 8, 'A49': 9}, 'A5': {}, 'A6': {'A61': 0, 'A62': 1, 'A63': 2, 'A64': 3, 'A65': 4}, 'A7': {'A71': 0, 'A72': 1, 'A73': 2, 'A74': 3, 'A75': 4}, 'A8': {}, 'A9': {'A91': 0, 'A92': 1, 'A93': 2, 'A94': 3}, 'A10': {'A101': 0, 'A102': 1, 'A103': 2}, 'A11': {}, 'A12': {'A121': 0, 'A122': 1, 'A123': 2, 'A124': 3}, 'A13': {}, 'A14': {'A141': 0, 'A142': 1, 'A143': 2}, 'A15': {'A151': 0, 'A152': 1, 'A153': 2}, 'A16': {}, 'A17': {'A171': 0, 'A172': 1, 'A173': 2, 'A174': 3}, 'A18': {}, 'A19': {'A191': 0, 'A192': 1}, 'A20': {'A201': 0, 'A202': 1}, 'Class': {'1': 0, '2': 1}}

Datos:
[[ 0  6  4 ...  1  0  0]
 [ 1 48  2 ... 

Probamos nuestra implementación de **Naive-Bayes** con validación siple con los datos de German.

In [15]:
    print("\n\nGerman:")
    print("\nValidacion Simple")
    print("\nValidando 100 veces con clasificador propio:")
    error = 0
    NB_laplace = 0.0
    NB_noLap = 0.0
    error_lap = 0
    for _ in range(0, 100):
        nb = ClasificadorNaiveBayes()
        vs = ValidacionSimple()
        ret = Clasificador.validacion(vs, german, nb)
        error += ret[0]
        error_lap += ret[1]
    NB_noLap = error / 100
    NB_laplace = error_lap / 100
    print("Error medio sin laplace " + str(NB_noLap))
    print("Error medio con laplace " + str(NB_laplace))




German:

Validacion Simple

Validando 100 veces con clasificador propio:
Error medio sin laplace 0.28269999999999995
Error medio con laplace 0.3020000000000001


Repetimos la prueba con el método *MultinomialNB* de sickit-learn, si bien en este caso no tiene much sentido porque trata todos los datos como nominales aún habiendo varios continuos.

In [16]:
    print("\nValidando 100 veces con MultinomialNB:")
    #En este caso no pretratamos los datos pues los transformaría todos en datos nominales y en este caso hay datos continuos
    atr = german.datos[:, :len(german.nominalAtributos) - 1]
    clase = german.datos[:, len(german.nominalAtributos) - 1]
    error = 0
    multi = 0.0
    for _ in range(0, 100):
        train_x, test_x, train_y, test_y = train_test_split(atr, clase, test_size=0.1)
        nb = MultinomialNB()
        nb.fit(train_x, train_y)
        res = nb.predict(test_x)
        error += error_sk(test_y, res)
    multi = error / 100
    print("Error medio " + str(multi))


Validando 100 veces con MultinomialNB:
Error medio 0.3628000000000002


Repetimos la prueba con el método *GaussianNB* de sickit-learn.

In [17]:
    print("\nValidando 100 veces con GaussianNB:")
    atr = german.datos[:, :len(german.nominalAtributos) - 1]
    clase = german.datos[:, len(german.nominalAtributos) - 1]
    error = 0
    gauss = 0.0
    for _ in range(0, 100):
        train_x, test_x, train_y, test_y = train_test_split(atr, clase, test_size=0.1)
        nb = GaussianNB()
        nb.fit(train_x, train_y)
        res = nb.predict(test_x)
        error += error_sk(test_y, res)
    gauss = error/100
    print("Error medio " + str(gauss))




Validando 100 veces con GaussianNB:
Error medio 0.2700999999999999


In [18]:
salida = "<table><tr><th>Clasificador</th><th>Nuestro</th><th>MultinomialNB</th><th>Gausssian</th></tr><tr><td>NB sin Laplace</td>"
salida += "<td>%f</td>" % (NB_noLap)
salida += "<td>-</td>" 
salida += "<td>-</td></tr>" 
salida += "<tr><td>NB con Laplace</td>"
salida += "<td>%f</td>" % (NB_laplace)
salida += "<td>%f</td>" % (multi)
salida += "<td>%f</td></tr></table>" % (gauss)

In [19]:
display(HTML(salida))

Clasificador,Nuestro,MultinomialNB,Gausssian
NB sin Laplace,0.282700,-,-
NB con Laplace,0.302000,0.362800,0.270100


Pasamos a hacer las pruebas con validación cruzada

In [20]:
    print("\nValidacion Cruzada")
    print("\nValidando con clasificador propio:")
    german = Datos("german.data")
    nb = ClasificadorNaiveBayes()
    vc = ValidacionCruzada()
    error = Clasificador.validacion(vc, german, nb)

    NB_noLap = error[0]
    NB_laplace = error[1]
    print("Error medio sin laplace " + str(error[0]))
    print("Error medio con laplace " + str(error[1]))



Validacion Cruzada

Validando con clasificador propio:
Error en partición 0 sin Laplace: 0.41
Error en partición 0 con Laplace: 0.39
Error en partición 1 sin Laplace: 0.26
Error en partición 1 con Laplace: 0.3
Error en partición 2 sin Laplace: 0.29
Error en partición 2 con Laplace: 0.28
Error en partición 3 sin Laplace: 0.23
Error en partición 3 con Laplace: 0.3
Error en partición 4 sin Laplace: 0.3
Error en partición 4 con Laplace: 0.33
Error en partición 5 sin Laplace: 0.24
Error en partición 5 con Laplace: 0.27
Error en partición 6 sin Laplace: 0.31
Error en partición 6 con Laplace: 0.3
Error en partición 7 sin Laplace: 0.3
Error en partición 7 con Laplace: 0.28
Error en partición 8 sin Laplace: 0.25
Error en partición 8 con Laplace: 0.3
Error en partición 9 sin Laplace: 0.31
Error en partición 9 con Laplace: 0.29
Error medio sin laplace 0.29
Error medio con laplace 0.304


Repetimos la prueba con el método *MultinomialNB* de sickit-learn.

In [21]:
    print("\nValidando con MultinomialNB:")
    #En este caso no pretratamos los datos pues los transformaría todos en datos nominales y en este caso hay datos continuos
    atr = german.datos[:, :len(german.nominalAtributos) - 1]
    clase = german.datos[:, len(german.nominalAtributos) - 1]
    acierto = cross_val_score(MultinomialNB(), atr, clase)
    error = []
    for data in acierto:
        error.append(1 - data)
    print("Error por particiones: ")
    print(error)
    multi = sum(error) / len(error)
    print("Error medio: " + str(multi))


Validando con MultinomialNB:
Error por particiones: 
[0.31499999999999995, 0.37, 0.32999999999999996, 0.41000000000000003, 0.375]
Error medio: 0.36


Repetimos la prueba con el método *GaussianNB* de sickit-learn.

In [22]:
    print("\nValidando con GaussianNB:")
    atr = german.datos[:, :len(german.nominalAtributos) - 1]
    clase = german.datos[:, len(german.nominalAtributos) - 1]
    acierto = cross_val_score(GaussianNB(), atr, clase)
    error = []
    for data in acierto:
        error.append(1 - data)
    print("Error por particiones: ")
    print(error)
    gauss = sum(error) / len(error)
    print("Error medio: " + str(gauss))




Validando con GaussianNB:
Error por particiones: 
[0.29500000000000004, 0.28, 0.24, 0.30500000000000005, 0.21999999999999997]
Error medio: 0.268


In [23]:
salida = "<table><tr><th>Clasificador</th><th>Nuestro</th><th>MultinomialNB</th><th>Gausssian</th></tr><tr><td>NB sin Laplace</td>"
salida += "<td>%f</td>" % (NB_noLap)
salida += "<td>-</td>" 
salida += "<td>-</td></tr>" 
salida += "<tr><td>NB con Laplace</td>"
salida += "<td>%f</td>" % (NB_laplace)
salida += "<td>%f</td>" % (multi)
salida += "<td>%f</td></tr></table>" % (gauss)

In [24]:
display(HTML(salida))

Clasificador,Nuestro,MultinomialNB,Gausssian
NB sin Laplace,0.290000,-,-
NB con Laplace,0.304000,0.360000,0.268000
